In [1]:
import time
import pandas as pd
import numpy as np
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

In [2]:
def init_driver():
    driver = webdriver.Chrome()
    driver.wait = WebDriverWait(driver, 3)
    return driver

In [3]:
def lookup(driver):
    post_dict = {'link':[], 'title':[], 'stats':[], 'last_post_stats':[]}
    driver.get('http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat')
    driver.find_element_by_xpath('''//*[@id="yui-gen11"]''').click()
    counter1 = 0
    page_number = 1

    while True:
        try:
            link = driver.find_element_by_link_text(str(page_number))
            link.click()
            print(driver.current_url)
            page_number += 1
            posts = driver.find_elements_by_xpath('''.//*[@id='threads']''')
            for post in posts:
                titles = post.find_elements_by_xpath('''.//a[@class='title']''')
                for title in titles:
                    #print(title.text)
                    if page_number > 13:
                        print(title.text)
                    post_dict['title'].append(title.text)
                    #print(title.get_attribute('href'))
                    post_dict['link'].append(title.get_attribute('href'))
                subs_stats = post.find_elements_by_xpath('''.//*[@class='threadstats td alt']''')
                for sub_stats in subs_stats:
                    #print(sub_stats.text)
                    post_dict['stats'].append(sub_stats.text)
                subs_dates = post.find_elements_by_xpath('''.//*[@class='threadlastpost td']''')
                for sub_dates in subs_dates:
                    #print(sub_dates.text)
                    post_dict['last_post_stats'].append(sub_dates.text)
                
        except NoSuchElementException:
            break

    return post_dict

In [4]:
def process_df(post_dict):
    post_dict['Last_post_date'] = post_dict.last_post_stats.apply(lambda x:(x.split(',')[0]).split('\n')[1])
    post_dict['Last_post_time'] = post_dict.last_post_stats.apply(lambda x: x.split(',')[1])
    
    post_dict.Last_post_date = post_dict.Last_post_date.apply(lambda x:datetime.datetime.strptime(x,'%m-%d-%Y') if x not in ('Yesterday','Today') else x)
    post_dict.Last_post_time = post_dict.Last_post_time.apply(lambda x:(time.strftime('%H:%M',time.strptime(x,' %I:%M %p'))))
                                                              
    post_dict['Replies'] = post_dict.stats.apply(lambda x: int((x.split('\n')[0]).split(':')[1].replace(',', '')))
    post_dict['Views'] = post_dict.stats.apply(lambda x: int(((x.split('\n')[1]).split(': ')[1]).replace(',','')))
    post_dict.drop(['last_post_stats', 'stats'], axis=1, inplace=True)
    
    return post_dict

In [5]:
def sort_values_by(df, by, ascending):
    sorted_df = df.sort_values(by=by, ascending=ascending)
    return sorted_df

In [6]:
driver = init_driver()
data = lookup(driver)
time.sleep(5)
driver.quit()

http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index2.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index3.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index4.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index5.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index6.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index7.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index8.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index9.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index10.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index11.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index12.html
http://www.f150ecoboost.net/forum/68-2016-ford-f150-ecoboost-chat/index13

In [7]:
data = pd.DataFrame.from_dict(data)

In [8]:
data = process_df(data)

In [9]:
data.tail()

,link,title,Last_post_date,Last_post_time,Replies,Views
261,http://www.f150ecoboost.net/forum/68-2016-ford...,New member,2016-01-27 00:00:00,11:20,8,601
262,http://www.f150ecoboost.net/forum/68-2016-ford...,"New member, purchasing first eco boost",2016-01-23 00:00:00,11:44,14,2234
263,http://www.f150ecoboost.net/forum/68-2016-ford...,New to the Boost,2016-01-15 00:00:00,05:55,6,960
264,http://www.f150ecoboost.net/forum/68-2016-ford...,This section of the forum seems lonely...,2015-12-29 00:00:00,12:12,26,3314
265,http://www.f150ecoboost.net/forum/68-2016-ford...,New Member with a 2016 F-150 3.5 Ecoboost.,2015-12-14 00:00:00,17:22,8,2133


In [10]:
data.dtypes

link              object
title             object
Last_post_date    object
Last_post_time    object
Replies            int64
Views              int64
dtype: object

In [11]:
sorted_data = sort_values_by(data, ['Replies','Views'], [False,False])

In [12]:
sorted_data.head()
type(sorted_data)

pandas.core.frame.DataFrame

In [13]:
a = sorted_data.groupby('Last_post_date').apply(pd.DataFrame.sort_values, 'Replies')

In [14]:
a.head()

link  \
Last_post_date                                                               
2015-12-14 00:00:00 252  http://www.f150ecoboost.net/forum/68-2016-ford...   
                    265  http://www.f150ecoboost.net/forum/68-2016-ford...   
2015-12-29 00:00:00 251  http://www.f150ecoboost.net/forum/68-2016-ford...   
                    264  http://www.f150ecoboost.net/forum/68-2016-ford...   
2016-01-15 00:00:00 250  http://www.f150ecoboost.net/forum/68-2016-ford...   

                                                              title  \
Last_post_date                                                        
2015-12-14 00:00:00 252  New Member with a 2016 F-150 3.5 Ecoboost.   
                    265  New Member with a 2016 F-150 3.5 Ecoboost.   
2015-12-29 00:00:00 251   This section of the forum seems lonely...   
                    264   This section of the forum seems lonely...   
2016-01-15 00:00:00 250                            New to the Boost   

                              Last_post_date Last_post_time  Replies  Views  
Last_post_date                                                               
2015-12-14 00:00:00 252  2015-12-14 00:00:00          17:22        8   2133  
                    265  2015-12-14 00:00:00          17:22        8   2133  
2015-12-29 00:00:00 251  2015-12-29 00:00:00          12:12       26   3314  
                    264  2015-12-29 00:00:00          12:12       26   3314  
2016-01-15 00:00:00 250  2016-01-15 00:00:00          05:55        6    960

In [15]:
type(a)

pandas.core.frame.DataFrame

In [16]:
a.to_csv('Top100_Posts.csv')

In [17]:
data.to_csv(('check.csv'))

In [18]:
data.shape

(266, 6)